# Parte 1

In [10]:
language = "pt"

In [11]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const b2text = blob => new Promise(resolve => {
    const reader = new FileReader();
    reader.onloadend = e => resolve(e.srcElement.result);
    reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);
    let chunks = [];
    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.onstop = async () => {
        const blob = new Blob(chunks, { type: 'audio/wav' });
        const b64data = await b2text(blob);
        resolve(b64data);
    };
    recorder.start();
    setTimeout(() => recorder.stop(), time);
});
"""

def record(sec=5):
    display(Javascript(RECORD))
    js_result = output.eval_js(f'record({sec*1000})')
    audio = b64decode(js_result.split(',')[1])
    file_name = 'audiozin.wav'
    with open(file_name, 'wb') as f:
        f.write(audio)
    return file_name
print('Ouvindo...\n')
record_file = record()
display(Audio(record_file))


Ouvindo...



<IPython.core.display.Javascript object>

# Parte 2

In [12]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import whisper
model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 Qual é a linguagem mais utilizada a gente a gente para a programação?


# Parte 3

In [14]:
!pip install openai

In [15]:
import os
os.environ['OPENAI_API_KEY'] = 'minhachave'

In [16]:
import openai
openai.api_key = os.environ.get('OPENAI_API_KEY')
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[ { "role": "user", "content": transcription } ]
)
chatgpt_response = response.choices[0].message.content
print(chatgpt_response)

Atualmente, a linguagem de programação mais utilizada pela maioria das pessoas é JavaScript. Essa linguagem é amplamente utilizada para o desenvolvimento de aplicativos web e móveis, além de ser uma linguagem muito versátil e compatível com praticamente todos os navegadores. Outras linguagens de programação populares incluem Python, Java, C++ e Ruby.


# 4 parte

In [17]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [18]:
from gtts import  gTTS
gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay=True))